In [16]:
from DataPersitenceService import DBMS
from backend import Notification
from datetime import datetime

In [ ]:
class BadgeHandler:
    def __init__(self):
        self.DBMS = DBMS()


    def check_for_badges(self):
        all_badges = self.DBMS.read("GET_ALL_BADGES",{})
        for bid in all_badges['bid'].values:
            self.check_badge_achievers(bid)


    def assign_badge(self, uid, bid):
        """
        Assign a badge to a user.

        :param uid: User ID.
        :param bid: Badge ID.
        :return: None. Awards the badge to the specified user.
        """
        # Award the badge to the user
        self.DBMS.write("AWARD_BADGE", {"_USER_ID_": uid, "_BADGE_ID_": bid})

    def get_current_date(self):
        # Get the current date
        current_date = datetime.now()

        # Format the date as yyyy-mm-dd
        formatted_date = current_date.strftime('%Y-%m-%d')

        return formatted_date
    
    def query_badge_notification_info(self,uid,bid):
        # Get badge name
        badge_name = self.DBMS.read('GET_BADGE_NAME',{'_BADGE_ID_':bid})['badge_name'].values[0]

        # get tournament info
        tournament = self.DBMS.read('GET_TOURNAMENT_NAME_FROM_BADGE_ID',{'_BADGE_ID_':bid})
        tournament_name = tournament['tournament_name'].values[0]
        tournament_id = tournament['tournament_id'].values[0]

        # get user info
        user_name = self.DBMS.read('GET_USER_NAME_FROM_UID',{'_USER_ID_':uid})['user_name'].values[0]

        notification_info = {uid:{"_BADGE_NAME_":badge_name,
                             '_USER_NAME_':user_name,
                             '_TOURNAMENT_NAME_':tournament_name,
                             '_TOURNAMENT_ID_': tournament_id,
                             '_BADGE_ACHIEVED_DATE_':self.get_current_date()}
                             }

        # return to notification
        return notification_info


    def badge_awarded_notification(self,uid,bid):

        notification_info = self.query_badge_notification_info(uid,bid)

        BadgeNotification = Notification('BADGE_ACHIEVED')

        BadgeNotification.register_notfications_to_messageboard(notification_info)


    def check_badge_achievers(self, bid):
        """
        Check and assign badges to eligible users based on badge criteria.

        :param bid: Badge ID.
        :return: None. Assigns badges to users who meet the criteria.
        """
        # Get badge logic
        badge_logic_df = self.DBMS.read("GET_BADGE_LOGIC", {"_BADGE_ID_": bid})

        badge_logic = {
            "_TOURNAMENT_ID_": badge_logic_df["tournament_id"].values[0],
            "_RANK_": badge_logic_df["rank"].values[0],
            "_NUM_BATTLES_": badge_logic_df["num_battles"].values[0],
        }

        # Get all potential badge achievers
        badge_achievers = self.DBMS.read("GET_BADGE_ACHIEVERS", badge_logic)[
            "uid"
        ].values

        # Get current badge holders
        existing_badge_achievers = self.DBMS.read(
            "GET_CURRENT_BADGE_HOLDERS", {"_BADGE_ID_": bid}
        )["uid"].values

        # Determine new badge awardees
        new_awardees = set(badge_achievers) - set(existing_badge_achievers)
        for _uid in new_awardees:
            
            # Register badge assignment in DB
            self.assign_badge(_uid, bid)

            # upload notification of badge assignment to messageboard
            self.badge_awarded_notification(_uid, bid)

In [20]:
DBMS_ = DBMS()

def get_current_date():
    # Get the current date
    current_date = datetime.now()

    # Format the date as yyyy-mm-dd
    formatted_date = current_date.strftime('%Y-%m-%d')

    return formatted_date

def query_badge_notification_info(bid,uid):
    # Get badge name
    badge_name = DBMS_.read('GET_BADGE_NAME',{'_BADGE_ID_':bid})['badge_name'].values[0]

    # get tournament info
    tournament = DBMS_.read('GET_TOURNAMENT_NAME_FROM_BADGE_ID',{'_BADGE_ID_':bid})
    tournament_name = tournament['tournament_name'].values[0]
    tournament_id = tournament['tournament_id'].values[0]

    # get user info
    user_name = DBMS_.read('GET_USER_NAME_FROM_UID',{'_USER_ID_':uid})['user_name'].values[0]

    notification_info = {uid:{"_BADGE_NAME_":badge_name,
                            '_USER_NAME_':user_name,
                            '_TOURNAMENT_NAME_':tournament_name,
                            '_TOURNAMENT_ID_': tournament_id,
                            '_BADGE_ACHIEVED_DATE_':get_current_date()}
                            }

    # return to notification
    return notification_info


def badge_awarded_notification(uid,bid):

    notification_info = query_badge_notification_info(uid,bid)

    BadgeNotification = Notification('BADGE_ACHIEVED')

    BadgeNotification.register_notfications_to_messageboard(notification_info)

def check_badge_achievers(bid):
        """
        Check and assign badges to eligible users based on badge criteria.

        :param bid: Badge ID.
        :return: None. Assigns badges to users who meet the criteria.
        """
        # Get badge logic
        badge_logic_df = DBMS_.read("GET_BADGE_LOGIC", {"_BADGE_ID_": bid})
        print(badge_logic_df)
        badge_logic = {
            "_TOURNAMENT_ID_": badge_logic_df["tournament_id"].values[0],
            "_RANK_": badge_logic_df["rank"].values[0],
            "_NUM_BATTLES_": badge_logic_df["num_battles"].values[0],
        }
        print(badge_logic)

        # Get all potential badge achievers
        badge_achievers = DBMS_.read("GET_BADGE_ACHIEVERS", badge_logic)[
            "uid"
        ].values

        print(badge_achievers)

        # Get current badge holders
        existing_badge_achievers = DBMS_.read(
            "GET_CURRENT_BADGE_HOLDERS", {"_BADGE_ID_": bid}
        )["uid"].values

        print(existing_badge_achievers)
        # Determine new badge awardees
        new_awardees = set(badge_achievers) - set(existing_badge_achievers)
        print(new_awardees)
        for _uid in new_awardees:
            
            # Register badge assignment in DB
            #self.assign_badge(_uid, bid)

            # upload notification of badge assignment to messageboard
            print(_uid)
            badge_awarded_notification(_uid, bid)

def check_for_badges():
    all_badges = DBMS_.read("GET_ALL_BADGES",{})
    for bid in all_badges['bid'].values:
        print(bid)#check_badge_achievers(bid)

In [21]:
check_badge_achievers(6)

   tournament_id  rank  num_battles
0             12     1            5
{'_TOURNAMENT_ID_': 12, '_RANK_': 1, '_NUM_BATTLES_': 5}
[5]
[]
{5}
5


IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
DBMS_.read('GET_USER_NAME_FROM_UID',{'_USER_ID_':5})['user_name'].values[0]

'AskeMeineche'